# 🎯 YOLOv11 Pose Estimation Lab: Time Series Analysis with Person Tracking

---

## 📋 Lab Overview

**วัตถุประสงค์การเรียนรู้ (Learning Objectives):**
1. เข้าใจการทำ Multi-Person Pose Estimation ด้วย YOLOv11
2. สร้างระบบ Person Tracking เพื่อรักษา Person ID ข้ามเฟรม
3. แปลงข้อมูล Pose เป็น Time Series Dataset
4. วิเคราะห์และสร้าง Dataset สำหรับบุคคลเฉพาะ (Specific Person ID)
5. Visualize และ Export ข้อมูลสำหรับการวิเคราะห์ต่อไป

**Prerequisites:**
- Python 3.8+
- GPU recommended (CUDA compatible)
- Basic understanding of computer vision

**Estimated Time:** 2-3 hours

---

## 📚 Part 1: Environment Setup

### 1.1 Install Required Packages

ติดตั้ง libraries ที่จำเป็นสำหรับ Pose Estimation และ Time Series Analysis

In [4]:
# =====================================================
# STEP 1.1: Install Required Libraries
# =====================================================
# Run these commands in terminal or uncomment to run in notebook

# !pip install ultralytics opencv-python pandas matplotlib seaborn numpy
# !pip install lap filterpy scikit-learn plotly
# !pip install ipywidgets tqdm

# print("📦 Please ensure the following packages are installed:")
# print("   - ultralytics (YOLOv11)")
# print("   - opencv-python (Video processing)")
# print("   - pandas (Data manipulation)")
# print("   - matplotlib, seaborn, plotly (Visualization)")
# print("   - lap, filterpy (Tracking algorithms)")

### 1.2 Import Libraries

นำเข้า libraries ทั้งหมดที่จำเป็นสำหรับ lab นี้

In [5]:
# =====================================================
# STEP 1.2: Import All Required Libraries
# =====================================================

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import json
import warnings
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from tqdm import tqdm

# YOLOv11
from ultralytics import YOLO

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"📅 Lab started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All libraries imported successfully!
📅 Lab started at: 2026-01-16 15:55:00


### 1.3 Define Constants and Configurations

กำหนดค่าคงที่สำหรับ COCO Keypoint format (17 keypoints)

In [6]:
# =====================================================
# STEP 1.3: Define Constants
# =====================================================

# COCO Keypoint Names (17 keypoints)
KEYPOINT_NAMES = [
    "nose",           # 0
    "left_eye",       # 1
    "right_eye",      # 2
    "left_ear",       # 3
    "right_ear",      # 4
    "left_shoulder",  # 5
    "right_shoulder", # 6
    "left_elbow",     # 7
    "right_elbow",    # 8
    "left_wrist",     # 9
    "right_wrist",    # 10
    "left_hip",       # 11
    "right_hip",      # 12
    "left_knee",      # 13
    "right_knee",     # 14
    "left_ankle",     # 15
    "right_ankle"     # 16
]

# Keypoint indices for body parts
BODY_PARTS = {
    'head': [0, 1, 2, 3, 4],
    'upper_body': [5, 6, 7, 8, 9, 10],
    'lower_body': [11, 12, 13, 14, 15, 16],
    'left_arm': [5, 7, 9],
    'right_arm': [6, 8, 10],
    'left_leg': [11, 13, 15],
    'right_leg': [12, 14, 16],
    'torso': [5, 6, 11, 12]
}

# COCO Skeleton Connections for visualization
SKELETON_CONNECTIONS = [
    # Head connections
    (0, 1), (0, 2), (1, 3), (2, 4),
    # Upper body
    (5, 6),   # shoulders
    (5, 7), (7, 9),    # left arm
    (6, 8), (8, 10),   # right arm
    # Torso
    (5, 11), (6, 12),  # shoulder to hip
    (11, 12),          # hips
    # Lower body
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16)   # right leg
]

print("✅ Constants defined:")
print(f"   📍 Total keypoints: {len(KEYPOINT_NAMES)}")
print(f"   🦴 Skeleton connections: {len(SKELETON_CONNECTIONS)}")
print(f"   🏃 Body parts defined: {list(BODY_PARTS.keys())}")

✅ Constants defined:
   📍 Total keypoints: 17
   🦴 Skeleton connections: 16
   🏃 Body parts defined: ['head', 'upper_body', 'lower_body', 'left_arm', 'right_arm', 'left_leg', 'right_leg', 'torso']


---

## 📚 Part 2: Load YOLOv11 Pose Model

โหลด Pre-trained YOLOv11 Pose Estimation Model

**Model Options:**
- `yolo11n-pose.pt` - Nano (เร็วที่สุด, เหมาะกับ real-time)
- `yolo11s-pose.pt` - Small
- `yolo11m-pose.pt` - Medium (สมดุล)
- `yolo11l-pose.pt` - Large
- `yolo11x-pose.pt` - Extra Large (แม่นยำที่สุด)

In [7]:
# =====================================================
# STEP 2.1: Load YOLOv11 Pose Model
# =====================================================

# Choose model size based on your needs
MODEL_NAME = 'yolo11x-pose.pt'  # Change to 'yolo11m-pose.pt' for better accuracy

print(f"🔄 Loading model: {MODEL_NAME}")
model = YOLO(MODEL_NAME)

print(f"✅ Model loaded successfully!")
print(f"   📊 Model: {MODEL_NAME}")
print(f"   🎯 Task: Pose Estimation")
print(f"   📍 Keypoints: {len(KEYPOINT_NAMES)}")

🔄 Loading model: yolo11x-pose.pt
✅ Model loaded successfully!
   📊 Model: yolo11x-pose.pt
   🎯 Task: Pose Estimation
   📍 Keypoints: 17


 #### Set Video Path and Process

In [8]:
import cv2
from IPython.display import display, Image, clear_output
import time

def display_video(video_path: str, width: int = 640, speed: float = 1.0, skip_frames: int = 0):
    """
    Display a video file in Jupyter Notebook with detailed progress bar.
    
    Parameters:
        video_path: Path to the video file
        width: Target width for display (maintains aspect ratio)
        speed: Playback speed multiplier
        skip_frames: Number of frames to skip between each displayed frame
                     (0 = show all frames, 1 = show every 2nd frame, 2 = show every 3rd frame, etc.)
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Cannot open video file: {video_path}")
        return
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if fps <= 0:
        fps = 30
    
    total_seconds = total_frames / fps
    delay = 1.0 / (fps * speed)
    
    # Progress bar settings
    bar_height = 20
    bar_color = (0, 200, 0)        # Green
    bar_bg_color = (40, 40, 40)    # Dark gray
    text_color = (255, 255, 255)   # White
    
    try:
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            
            frame_count += 1
            
            # Skip frames if needed
            if skip_frames > 0 and (frame_count - 1) % (skip_frames + 1) != 0:
                continue
            
            # Resize frame maintaining aspect ratio
            h, w = frame.shape[:2]
            aspect_ratio = h / w
            target_height = int(width * aspect_ratio)
            frame = cv2.resize(frame, (width, target_height))
            
            # Calculate time
            current_seconds = frame_count / fps
            current_time = f"{int(current_seconds // 60):02d}:{int(current_seconds % 60):02d}"
            total_time = f"{int(total_seconds // 60):02d}:{int(total_seconds % 60):02d}"
            
            # Draw progress bar background
            cv2.rectangle(frame, 
                          (0, target_height - bar_height), 
                          (width, target_height), 
                          bar_bg_color, -1)
            
            # Draw progress bar
            progress = frame_count / total_frames
            progress_width = int(width * progress)
            cv2.rectangle(frame, 
                          (0, target_height - bar_height), 
                          (progress_width, target_height), 
                          bar_color, -1)
            
            # Draw time text
            time_text = f"{current_time} / {total_time}"
            cv2.putText(frame, time_text, 
                        (10, target_height - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, text_color, 1)
            
            # Draw speed and skip info
            info_text = ""
            if speed != 1.0:
                info_text += f"{speed}x"
            if skip_frames > 0:
                if info_text:
                    info_text += " | "
                info_text += f"skip:{skip_frames}"
            
            if info_text:
                text_size = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                cv2.putText(frame, info_text, 
                            (width - text_size[0] - 10, target_height - 5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.5, text_color, 1)
            
            # Convert to JPEG and display
            _, buffer = cv2.imencode('.jpg', frame)
            display(Image(data=buffer.tobytes()))
            clear_output(wait=True)
            
            time.sleep(delay)
                
    except KeyboardInterrupt:
        print("Playback interrupted.")
    finally:
        cap.release()
        print("Video stream ended.")

In [9]:
# =====================================================
# STEP  Configure and Process Video
# =====================================================

# ⚠️ IMPORTANT: Change this to your video path!
VIDEO_PATH = "./muay_thai_clip.mp4"  # 👈 แก้ไขตรงนี้

# Output paths
OUTPUT_DIR = Path("pose_output")
OUTPUT_DIR.mkdir(exist_ok=True)

OUTPUT_VIDEO = str(OUTPUT_DIR / "annotated_video.mp4")
OUTPUT_JSON = str(OUTPUT_DIR / "pose_data.json")
OUTPUT_CSV = str(OUTPUT_DIR / "pose_timeseries.csv")

print(f"📁 Output directory: {OUTPUT_DIR.absolute()}")

📁 Output directory: /home/student/workspace/DL-FOR-COMPUTER-VISION/week06/02_Advance/pose_output


In [10]:
# Show every 5th frame (skip 4)
display_video(VIDEO_PATH, width=1024, speed=1.0, skip_frames=8)

Video stream ended.


---

## 📚 Part 3: Person Tracker Class

### 3.1 Understanding Person Tracking

**ปัญหา:** เมื่อ detect pose ในแต่ละเฟรม เราจะได้ผลลัพธ์ที่ไม่มี ID 
ทำให้ไม่สามารถติดตามบุคคลเดิมข้ามเฟรมได้

**วิธีแก้:** ใช้ Centroid Tracking Algorithm
1. คำนวณ centroid ของแต่ละคนจาก keypoints
2. เปรียบเทียบ centroid กับเฟรมก่อนหน้า
3. Assign ID ให้กับ detection ที่ใกล้ที่สุด

In [11]:
# =====================================================
# STEP 3.1: Create Enhanced Person Tracker Class
# =====================================================

@dataclass
class PersonTrackInfo:
    """Store tracking information for each person"""
    person_id: int
    centroid: np.ndarray
    bbox: Optional[np.ndarray] = None
    keypoints: Optional[np.ndarray] = None
    confidence: float = 0.0
    frame_count: int = 0
    disappeared_count: int = 0
    history: List[np.ndarray] = field(default_factory=list)


class EnhancedPersonTracker:
    """
    Advanced centroid-based tracker for maintaining person IDs across frames
    
    Features:
    - Centroid-based tracking with velocity prediction
    - Handles occlusion and temporary disappearance
    - Maintains tracking history for each person
    """
    
    def __init__(self, 
                 max_disappeared: int = 30,
                 max_distance: float = 100.0,
                 min_confidence: float = 0.3,
                 use_velocity: bool = True):
        """
        Initialize the tracker
        
        Args:
            max_disappeared: Maximum frames before removing a track
            max_distance: Maximum distance threshold for matching
            min_confidence: Minimum detection confidence
            use_velocity: Whether to use velocity prediction
        """
        self.next_person_id = 1
        self.tracks: Dict[int, PersonTrackInfo] = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        self.min_confidence = min_confidence
        self.use_velocity = use_velocity
        
        # Statistics
        self.total_persons_detected = 0
        self.frame_count = 0
        
    def _calculate_centroid(self, keypoints: np.ndarray) -> np.ndarray:
        """Calculate centroid from valid keypoints"""
        valid_mask = (keypoints[:, 0] > 0) & (keypoints[:, 1] > 0)
        if np.sum(valid_mask) > 0:
            return keypoints[valid_mask].mean(axis=0)
        return np.array([0, 0])
    
    def _predict_position(self, track: PersonTrackInfo) -> np.ndarray:
        """Predict next position using velocity"""
        if not self.use_velocity or len(track.history) < 2:
            return track.centroid
        
        # Calculate velocity from last two positions
        velocity = track.history[-1] - track.history[-2]
        return track.centroid + velocity
    
    def register(self, centroid: np.ndarray, keypoints: np.ndarray, 
                 confidence: float) -> int:
        """Register a new person with unique ID"""
        person_id = self.next_person_id
        self.tracks[person_id] = PersonTrackInfo(
            person_id=person_id,
            centroid=centroid,
            keypoints=keypoints.copy(),
            confidence=confidence,
            frame_count=1,
            disappeared_count=0,
            history=[centroid.copy()]
        )
        self.next_person_id += 1
        self.total_persons_detected += 1
        return person_id
    
    def deregister(self, person_id: int):
        """Remove person from active tracking"""
        if person_id in self.tracks:
            del self.tracks[person_id]
    
    def update(self, detections: List[Dict]) -> Dict[int, int]:
        """
        Update tracker with new detections
        
        Args:
            detections: List of dicts with 'keypoints', 'confidence'
            
        Returns:
            Dict mapping person_id to detection_index
        """
        self.frame_count += 1
        
        # If no detections, mark all as disappeared
        if len(detections) == 0:
            for person_id in list(self.tracks.keys()):
                self.tracks[person_id].disappeared_count += 1
                if self.tracks[person_id].disappeared_count > self.max_disappeared:
                    self.deregister(person_id)
            return {}
        
        # Calculate centroids for all detections
        detection_centroids = []
        for det in detections:
            centroid = self._calculate_centroid(det['keypoints'])
            detection_centroids.append(centroid)
        detection_centroids = np.array(detection_centroids)
        
        # If no existing tracks, register all detections
        if len(self.tracks) == 0:
            assignments = {}
            for idx, det in enumerate(detections):
                if det['confidence'] >= self.min_confidence:
                    person_id = self.register(
                        detection_centroids[idx],
                        det['keypoints'],
                        det['confidence']
                    )
                    assignments[person_id] = idx
            return assignments
        
        # Get existing track positions (with velocity prediction)
        person_ids = list(self.tracks.keys())
        predicted_positions = np.array([
            self._predict_position(self.tracks[pid]) for pid in person_ids
        ])
        
        # Calculate distance matrix
        distances = np.linalg.norm(
            predicted_positions[:, np.newaxis] - detection_centroids,
            axis=2
        )
        
        # Hungarian algorithm-like matching (greedy for simplicity)
        assignments = {}
        used_detections = set()
        used_tracks = set()
        
        # Sort by distance and assign
        flat_indices = np.argsort(distances.flatten())
        for flat_idx in flat_indices:
            track_idx = flat_idx // len(detections)
            det_idx = flat_idx % len(detections)
            
            if track_idx in used_tracks or det_idx in used_detections:
                continue
            
            if distances[track_idx, det_idx] > self.max_distance:
                continue
            
            if detections[det_idx]['confidence'] < self.min_confidence:
                continue
            
            person_id = person_ids[track_idx]
            assignments[person_id] = det_idx
            used_detections.add(det_idx)
            used_tracks.add(track_idx)
            
            # Update track
            track = self.tracks[person_id]
            track.centroid = detection_centroids[det_idx]
            track.keypoints = detections[det_idx]['keypoints'].copy()
            track.confidence = detections[det_idx]['confidence']
            track.frame_count += 1
            track.disappeared_count = 0
            track.history.append(detection_centroids[det_idx].copy())
            
            # Keep history limited
            if len(track.history) > 30:
                track.history = track.history[-30:]
        
        # Handle unmatched tracks (disappeared)
        for track_idx, person_id in enumerate(person_ids):
            if track_idx not in used_tracks:
                self.tracks[person_id].disappeared_count += 1
                if self.tracks[person_id].disappeared_count > self.max_disappeared:
                    self.deregister(person_id)
        
        # Register new detections
        for det_idx, det in enumerate(detections):
            if det_idx not in used_detections:
                if det['confidence'] >= self.min_confidence:
                    person_id = self.register(
                        detection_centroids[det_idx],
                        det['keypoints'],
                        det['confidence']
                    )
                    assignments[person_id] = det_idx
        
        return assignments
    
    def get_active_tracks(self) -> Dict[int, PersonTrackInfo]:
        """Get all currently active tracks"""
        return self.tracks.copy()
    
    def get_statistics(self) -> Dict:
        """Get tracking statistics"""
        return {
            'total_persons_detected': self.total_persons_detected,
            'current_active_tracks': len(self.tracks),
            'frames_processed': self.frame_count,
            'active_person_ids': list(self.tracks.keys())
        }

print("✅ EnhancedPersonTracker class created!")
print("   Features:")
print("   - Velocity-based position prediction")
print("   - Occlusion handling")
print("   - Confidence filtering")
print("   - Tracking history maintenance")

✅ EnhancedPersonTracker class created!
   Features:
   - Velocity-based position prediction
   - Occlusion handling
   - Confidence filtering
   - Tracking history maintenance


---

## 📚 Part 4: Video Processing Pipeline

### 4.1 Skeleton Visualization Function

ฟังก์ชันสำหรับวาด Skeleton บนเฟรม พร้อมแสดง Person ID

In [12]:
# =====================================================
# STEP 4.1: Skeleton Visualization Function
# =====================================================

def draw_skeleton(frame: np.ndarray, 
                  keypoints: np.ndarray, 
                  color: Tuple[int, int, int] = (0, 255, 0),
                  person_id: Optional[int] = None,
                  line_thickness: int = 2,
                  point_radius: int = 4,
                  show_keypoint_names: bool = False) -> np.ndarray:
    """
    Draw pose skeleton on frame
    
    Args:
        frame: Input image
        keypoints: Array of shape (17, 2) with (x, y) coordinates
        color: BGR color tuple
        person_id: Optional person ID to display
        line_thickness: Thickness of skeleton lines
        point_radius: Radius of keypoint circles
        show_keypoint_names: Whether to show keypoint names
    
    Returns:
        Frame with skeleton drawn
    """
    frame = frame.copy()
    
    # Draw connections
    for connection in SKELETON_CONNECTIONS:
        pt1_idx, pt2_idx = connection
        pt1 = keypoints[pt1_idx]
        pt2 = keypoints[pt2_idx]
        
        if pt1[0] > 0 and pt1[1] > 0 and pt2[0] > 0 and pt2[1] > 0:
            cv2.line(frame, 
                    (int(pt1[0]), int(pt1[1])), 
                    (int(pt2[0]), int(pt2[1])), 
                    color, line_thickness)
    
    # Draw keypoints
    for idx, kpt in enumerate(keypoints):
        if kpt[0] > 0 and kpt[1] > 0:
            cv2.circle(frame, (int(kpt[0]), int(kpt[1])), 
                      point_radius, color, -1)
            
            if show_keypoint_names:
                cv2.putText(frame, KEYPOINT_NAMES[idx][:3],
                           (int(kpt[0]) + 5, int(kpt[1]) - 5),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
    
    # Draw person ID label
    if person_id is not None:
        head_pt = keypoints[0]  # nose
        if head_pt[0] > 0 and head_pt[1] > 0:
            # Draw background rectangle
            label = f"ID: {person_id}"
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            cv2.rectangle(frame, 
                         (int(head_pt[0]) - 5, int(head_pt[1]) - h - 15),
                         (int(head_pt[0]) + w + 5, int(head_pt[1]) - 5),
                         color, -1)
            cv2.putText(frame, label, 
                       (int(head_pt[0]), int(head_pt[1]) - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return frame


def get_color_for_id(person_id: int, num_colors: int = 20) -> Tuple[int, int, int]:
    """Generate consistent color for each person ID"""
    colors = plt.cm.rainbow(np.linspace(0, 1, num_colors))
    color = colors[person_id % num_colors]
    # Convert to BGR and scale to 0-255
    return (int(color[2] * 255), int(color[1] * 255), int(color[0] * 255))

print("✅ Visualization functions created!")

✅ Visualization functions created!


### 4.2 Main Video Processing Function

ฟังก์ชันหลักสำหรับประมวลผลวิดีโอ พร้อมสร้าง Pose Data

In [13]:
# =====================================================
# STEP 4.2: Main Video Processing Function
# =====================================================

def process_video_with_pose_tracking(
    video_path: str,
    output_video_path: Optional[str] = None,
    conf_threshold: float = 0.3,
    max_disappeared: int = 30,
    show_progress: bool = True,
    save_every_n_frames: int = 1
) -> Tuple[List[Dict], Dict]:
    """
    Process video with pose estimation and person tracking
    
    Args:
        video_path: Path to input video file
        output_video_path: Path to save annotated video (optional)
        conf_threshold: Confidence threshold for detections
        max_disappeared: Max frames before removing track
        show_progress: Show progress bar
        save_every_n_frames: Save data every N frames (for memory efficiency)
    
    Returns:
        pose_data: List of frame data with pose information
        video_info: Dictionary with video metadata
    """
    # Open video
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {video_path}")
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0
    
    video_info = {
        'path': video_path,
        'fps': fps,
        'width': width,
        'height': height,
        'total_frames': total_frames,
        'duration_seconds': duration,
        'processed_at': datetime.now().isoformat()
    }
    
    print("=" * 60)
    print("📹 VIDEO INFORMATION")
    print("=" * 60)
    print(f"   Path: {video_path}")
    print(f"   Resolution: {width} x {height}")
    print(f"   FPS: {fps}")
    print(f"   Total Frames: {total_frames}")
    print(f"   Duration: {duration:.2f} seconds")
    print("=" * 60)
    
    # Initialize video writer
    out = None
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        print(f"📝 Output video will be saved to: {output_video_path}")
    
    # Initialize tracker
    tracker = EnhancedPersonTracker(
        max_disappeared=max_disappeared,
        max_distance=min(width, height) * 0.15,  # 15% of frame dimension
        min_confidence=conf_threshold
    )
    
    # Storage for pose data
    pose_data = []
    
    # Progress bar
    if show_progress:
        pbar = tqdm(total=total_frames, desc="Processing frames")
    
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLOv11 pose estimation
        results = model(frame, conf=conf_threshold, verbose=False)
        
        # Initialize frame data
        frame_data = {
            'frame_idx': frame_idx,
            'timestamp': frame_idx / fps if fps > 0 else 0,
            'persons': []
        }
        
        # Process detections
        if len(results) > 0 and results[0].keypoints is not None:
            keypoints_data = results[0].keypoints
            
            if keypoints_data.xy is not None and len(keypoints_data.xy) > 0:
                keypoints = keypoints_data.xy.cpu().numpy()  # [N, 17, 2]
                confidences = keypoints_data.conf.cpu().numpy()  # [N, 17]
                
                # Prepare detections for tracker
                detections = []
                for i in range(len(keypoints)):
                    avg_conf = np.mean(confidences[i][confidences[i] > 0])
                    detections.append({
                        'keypoints': keypoints[i],
                        'confidence': avg_conf if not np.isnan(avg_conf) else 0.0
                    })
                
                # Update tracker
                assignments = tracker.update(detections)
                
                # Store data for each tracked person
                for person_id, det_idx in assignments.items():
                    person_info = {
                        'person_id': person_id,
                        'keypoints': keypoints[det_idx].tolist(),
                        'keypoint_confidences': confidences[det_idx].tolist(),
                        'avg_confidence': float(detections[det_idx]['confidence'])
                    }
                    frame_data['persons'].append(person_info)
                    
                    # Draw skeleton on output frame
                    if out is not None:
                        color = get_color_for_id(person_id)
                        frame = draw_skeleton(frame, keypoints[det_idx], color, person_id)
        
        # Save frame data
        if frame_idx % save_every_n_frames == 0:
            pose_data.append(frame_data)
        
        # Write annotated frame
        if out is not None:
            # Add overlay information
            cv2.putText(frame, f"Frame: {frame_idx}/{total_frames}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Persons: {len(frame_data['persons'])}", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Time: {frame_data['timestamp']:.2f}s", 
                       (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            out.write(frame)
        
        frame_idx += 1
        if show_progress:
            pbar.update(1)
    
    # Cleanup
    cap.release()
    if out is not None:
        out.release()
    if show_progress:
        pbar.close()
    
    # Get final statistics
    stats = tracker.get_statistics()
    video_info['tracking_stats'] = stats
    
    print("\n" + "=" * 60)
    print("✅ PROCESSING COMPLETE")
    print("=" * 60)
    print(f"   Frames processed: {frame_idx}")
    print(f"   Total unique persons: {stats['total_persons_detected']}")
    print(f"   Data points saved: {len(pose_data)}")
    print("=" * 60)
    
    return pose_data, video_info

print("✅ Video processing function created!")

✅ Video processing function created!



## Create DataFrame from Pose Data

แปลงข้อมูล pose เป็น pandas DataFrame สำหรับการวิเคราะห์

In [14]:
# =====================================================
# STEP 6.1: Convert Pose Data to DataFrame
# =====================================================

def pose_data_to_dataframe(pose_data: List[Dict]) -> pd.DataFrame:
    """
    Convert pose data to a flat DataFrame for time series analysis
    
    Args:
        pose_data: List of frame data from video processing
    
    Returns:
        DataFrame with columns for each keypoint coordinate
    """
    records = []
    
    for frame_data in pose_data:
        frame_idx = frame_data['frame_idx']
        timestamp = frame_data['timestamp']
        
        for person in frame_data['persons']:
            record = {
                'frame_idx': frame_idx,
                'timestamp': timestamp,
                'person_id': person['person_id'],
                'avg_confidence': person['avg_confidence']
            }
            
            # Add each keypoint as separate columns
            keypoints = person['keypoints']
            confidences = person['keypoint_confidences']
            
            for kpt_idx, kpt_name in enumerate(KEYPOINT_NAMES):
                record[f'{kpt_name}_x'] = keypoints[kpt_idx][0]
                record[f'{kpt_name}_y'] = keypoints[kpt_idx][1]
                record[f'{kpt_name}_conf'] = confidences[kpt_idx]
            
            records.append(record)
    
    df = pd.DataFrame(records)
    
    if len(df) > 0:
        # Sort by frame and person_id
        df = df.sort_values(['frame_idx', 'person_id']).reset_index(drop=True)
    
    return df



---

## 📚 Part 5: Process Your Video

### 5.1 Set Video Path and Process

🔧 **แก้ไข VIDEO_PATH ให้ชี้ไปยังไฟล์วิดีโอของคุณ**

In [15]:
# =====================================================
# STEP 5.2: Process the Video
# =====================================================

# Process video with tracking
pose_data, video_info = process_video_with_pose_tracking(
    video_path=VIDEO_PATH,
    output_video_path=OUTPUT_VIDEO,
    conf_threshold=0.3,
    max_disappeared=35,
    show_progress=True,
    save_every_n_frames=1  # Save every frame
)

📹 VIDEO INFORMATION
   Path: ./muay_thai_clip.mp4
   Resolution: 1920 x 1080
   FPS: 29
   Total Frames: 3866
   Duration: 133.31 seconds
📝 Output video will be saved to: pose_output/annotated_video.mp4


Processing frames: 100%|██████████| 3866/3866 [02:12<00:00, 29.21it/s]


✅ PROCESSING COMPLETE
   Frames processed: 3866
   Total unique persons: 36
   Data points saved: 3866


# Display  Process  video

In [16]:
# Show every 5th frame (skip 4)
display_video(OUTPUT_VIDEO, width=1024, speed=1.0, skip_frames=8)

Video stream ended.


In [17]:
# Convert to DataFrame
df_poses = pose_data_to_dataframe(pose_data)
df_poses.head(10)

,frame_idx,timestamp,person_id,avg_confidence,nose_x,nose_y,nose_conf,left_eye_x,left_eye_y,left_eye_conf,...,left_knee_conf,right_knee_x,right_knee_y,right_knee_conf,left_ankle_x,left_ankle_y,left_ankle_conf,right_ankle_x,right_ankle_y,right_ankle_conf
0,0,0.000000,1,0.939729,1570.841064,628.137329,0.989106,1579.320312,621.487305,0.991083,...,0.997798,1542.994385,847.923523,0.997359,1597.515137,943.138306,0.990762,1544.355225,926.163391,0.990130
1,0,0.000000,2,0.857351,1391.924805,541.764343,0.983361,1398.174927,537.000183,0.991576,...,0.998740,1391.394531,713.093384,0.992637,1429.300781,784.686035,0.997721,1389.803467,773.052124,0.991802
2,0,0.000000,3,0.877679,1112.941162,528.165833,0.985352,1119.043701,523.015991,0.990616,...,0.998375,1112.048828,700.252930,0.993971,1146.626831,774.413330,0.996617,1112.166382,761.413574,0.991508
3,0,0.000000,4,0.883166,915.781311,563.037415,0.987232,923.174805,557.466187,0.992261,...,0.997968,915.727478,763.843018,0.989884,947.519165,849.709167,0.996019,917.615662,825.769226,0.987224
4,0,0.000000,5,0.714610,292.317749,626.045166,0.435317,294.536560,619.495117,0.544087,...,0.995617,325.843323,836.402527,0.973125,349.057526,931.232117,0.988034,330.143982,914.861084,0.956085
5,0,0.000000,6,0.875425,1216.902344,630.408752,0.983661,1226.102661,624.513306,0.990694,...,0.998719,1223.226807,846.422974,0.992903,1254.355591,935.658875,0.997297,1222.428833,916.559692,0.990620
6,0,0.000000,7,0.827648,681.639282,620.119873,0.985977,690.178955,613.558228,0.994562,...,0.997983,696.428894,837.792603,0.977912,729.912354,934.158691,0.996591,697.518921,911.403625,0.980470
7,0,0.000000,8,0.880806,442.516907,517.974365,0.968786,449.044312,512.601196,0.976328,...,0.996756,460.644928,700.089478,0.995145,483.259521,777.229980,0.988850,464.662811,765.981567,0.986470
8,0,0.000000,9,0.763042,748.287720,539.911499,0.856901,752.737183,534.797363,0.924724,...,0.996378,756.752808,706.454468,0.965748,791.612183,772.722168,0.992020,758.122253,761.784668,0.959178
9,1,0.034483,1,0.916054,1565.804810,627.088135,0.971860,1573.979736,621.207153,0.975588,...,0.998015,1543.400757,846.356323,0.997500,1597.720703,943.021362,0.991183,1544.024170,923.630615,0.990359


In [18]:
df_poses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34884 entries, 0 to 34883
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   frame_idx            34884 non-null  int64  
 1   timestamp            34884 non-null  float64
 2   person_id            34884 non-null  int64  
 3   avg_confidence       34884 non-null  float64
 4   nose_x               34884 non-null  float64
 5   nose_y               34884 non-null  float64
 6   nose_conf            34884 non-null  float64
 7   left_eye_x           34884 non-null  float64
 8   left_eye_y           34884 non-null  float64
 9   left_eye_conf        34884 non-null  float64
 10  right_eye_x          34884 non-null  float64
 11  right_eye_y          34884 non-null  float64
 12  right_eye_conf       34884 non-null  float64
 13  left_ear_x           34884 non-null  float64
 14  left_ear_y           34884 non-null  float64
 15  left_ear_conf        34884 non-null 

---

## 📚 Part 8: Create Time Series Pose Dataset with Labels

### 8.1 Define Action Labels for Frame Ranges

กำหนด Label สำหรับช่วง Frame ต่างๆ ของท่ามวยไทย

In [19]:
# =====================================================
# STEP 8.1: Define Action Labels for Frame Ranges
# =====================================================
import pprint


# Define action labels with frame ranges
ACTION_LABELS = {
    "Mek_Khara_Lor_Kaew": {"start_frame": 0, "end_frame": 461},
    "Kum_PaGUN_Poong_Hork": {"start_frame": 466, "end_frame": 1268},
    "Narai_Kwang_Jug": {"start_frame": 1302, "end_frame": 2030},
    "Hoang_Hern": {"start_frame": 2034, "end_frame": 2815},
    "PhraRam_Phangsorn": {"start_frame": 2832, "end_frame": 3335},

}

pprint.pprint(ACTION_LABELS)

{'Hoang_Hern': {'end_frame': 2815, 'start_frame': 2034},
 'Kum_PaGUN_Poong_Hork': {'end_frame': 1268, 'start_frame': 466},
 'Mek_Khara_Lor_Kaew': {'end_frame': 461, 'start_frame': 0},
 'Narai_Kwang_Jug': {'end_frame': 2030, 'start_frame': 1302},
 'PhraRam_Phangsorn': {'end_frame': 3335, 'start_frame': 2832}}


In [20]:
# =====================================================
# Vectorized approach using pd.cut or np.select
# =====================================================

import numpy as np

# Create conditions and choices for np.select
conditions = []
choices = []

for action_name, frame_range in ACTION_LABELS.items():
    condition = (df_poses['frame_idx'] >= frame_range['start_frame']) & \
                (df_poses['frame_idx'] <= frame_range['end_frame'])
    conditions.append(condition)
    choices.append(action_name)

# Apply labels
df_poses['action'] = np.select(conditions, choices, default='Unknown')

# Verify
print("✅ Action labels added!")
print(df_poses['action'].value_counts())

✅ Action labels added!
action
Kum_PaGUN_Poong_Hork    7229
Hoang_Hern              7056
Narai_Kwang_Jug         6591
Unknown                 5282
PhraRam_Phangsorn       4551
Mek_Khara_Lor_Kaew      4175
Name: count, dtype: int64


In [22]:
df_poses.head(10)

,frame_idx,timestamp,person_id,avg_confidence,nose_x,nose_y,nose_conf,left_eye_x,left_eye_y,left_eye_conf,...,right_knee_x,right_knee_y,right_knee_conf,left_ankle_x,left_ankle_y,left_ankle_conf,right_ankle_x,right_ankle_y,right_ankle_conf,action
0,0,0.000000,1,0.939729,1570.841064,628.137329,0.989106,1579.320312,621.487305,0.991083,...,1542.994385,847.923523,0.997359,1597.515137,943.138306,0.990762,1544.355225,926.163391,0.990130,Mek_Khara_Lor_Kaew
1,0,0.000000,2,0.857351,1391.924805,541.764343,0.983361,1398.174927,537.000183,0.991576,...,1391.394531,713.093384,0.992637,1429.300781,784.686035,0.997721,1389.803467,773.052124,0.991802,Mek_Khara_Lor_Kaew
2,0,0.000000,3,0.877679,1112.941162,528.165833,0.985352,1119.043701,523.015991,0.990616,...,1112.048828,700.252930,0.993971,1146.626831,774.413330,0.996617,1112.166382,761.413574,0.991508,Mek_Khara_Lor_Kaew
3,0,0.000000,4,0.883166,915.781311,563.037415,0.987232,923.174805,557.466187,0.992261,...,915.727478,763.843018,0.989884,947.519165,849.709167,0.996019,917.615662,825.769226,0.987224,Mek_Khara_Lor_Kaew
4,0,0.000000,5,0.714610,292.317749,626.045166,0.435317,294.536560,619.495117,0.544087,...,325.843323,836.402527,0.973125,349.057526,931.232117,0.988034,330.143982,914.861084,0.956085,Mek_Khara_Lor_Kaew
5,0,0.000000,6,0.875425,1216.902344,630.408752,0.983661,1226.102661,624.513306,0.990694,...,1223.226807,846.422974,0.992903,1254.355591,935.658875,0.997297,1222.428833,916.559692,0.990620,Mek_Khara_Lor_Kaew
6,0,0.000000,7,0.827648,681.639282,620.119873,0.985977,690.178955,613.558228,0.994562,...,696.428894,837.792603,0.977912,729.912354,934.158691,0.996591,697.518921,911.403625,0.980470,Mek_Khara_Lor_Kaew
7,0,0.000000,8,0.880806,442.516907,517.974365,0.968786,449.044312,512.601196,0.976328,...,460.644928,700.089478,0.995145,483.259521,777.229980,0.988850,464.662811,765.981567,0.986470,Mek_Khara_Lor_Kaew
8,0,0.000000,9,0.763042,748.287720,539.911499,0.856901,752.737183,534.797363,0.924724,...,756.752808,706.454468,0.965748,791.612183,772.722168,0.992020,758.122253,761.784668,0.959178,Mek_Khara_Lor_Kaew
9,1,0.034483,1,0.916054,1565.804810,627.088135,0.971860,1573.979736,621.207153,0.975588,...,1543.400757,846.356323,0.997500,1597.720703,943.021362,0.991183,1544.024170,923.630615,0.990359,Mek_Khara_Lor_Kaew


In [23]:
df_poses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34884 entries, 0 to 34883
Data columns (total 56 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   frame_idx            34884 non-null  int64  
 1   timestamp            34884 non-null  float64
 2   person_id            34884 non-null  int64  
 3   avg_confidence       34884 non-null  float64
 4   nose_x               34884 non-null  float64
 5   nose_y               34884 non-null  float64
 6   nose_conf            34884 non-null  float64
 7   left_eye_x           34884 non-null  float64
 8   left_eye_y           34884 non-null  float64
 9   left_eye_conf        34884 non-null  float64
 10  right_eye_x          34884 non-null  float64
 11  right_eye_y          34884 non-null  float64
 12  right_eye_conf       34884 non-null  float64
 13  left_ear_x           34884 non-null  float64
 14  left_ear_y           34884 non-null  float64
 15  left_ear_conf        34884 non-null 

In [24]:
df_poses.to_csv('pose_data.csv', index=False)

In [27]:
df_poses['action'].value_counts()

action
Kum_PaGUN_Poong_Hork    7229
Hoang_Hern              7056
Narai_Kwang_Jug         6591
Unknown                 5282
PhraRam_Phangsorn       4551
Mek_Khara_Lor_Kaew      4175
Name: count, dtype: int64

## DataFrame: `df_pose` - ข้อมูล Time Series จากการประมาณท่าทาง YOLOv11

### ภาพรวม
DataFrame นี้ประกอบด้วยข้อมูลการประมาณท่าทาง (Pose Estimation) หลายคนที่สกัดจากวิดีโอโดยใช้ YOLOv11 พร้อมการติดตามบุคคลข้ามเฟรม แต่ละแถวแทนบุคคลหนึ่งคนที่ตรวจจับได้ในหนึ่งเฟรม ประกอบด้วยพิกัด keypoint 17 จุดตามรูปแบบ COCO

### คำอธิบายคอลัมน์

**คอลัมน์ข้อมูลเมตา (Metadata):**

| คอลัมน์ | ชนิดข้อมูล | คำอธิบาย |
|--------|-------|-------------|
| `frame_idx` | int64 | หมายเลขเฟรมในลำดับวิดีโอ (เริ่มจาก 0) ใช้เป็นดัชนีเวลาหลักสำหรับการวิเคราะห์ time series |
| `timestamp` | float64 | ตำแหน่งเวลาเป็นวินาทีจากจุดเริ่มต้นวิดีโอ คำนวณจาก frame_idx / fps มีประโยชน์สำหรับการซิงโครไนซ์แบบ real-time และการวิเคราะห์เชิงเวลา |
| `person_id` | int64 | รหัสประจำตัวเฉพาะที่กำหนดโดยตัวติดตามบุคคล (person tracker) เพื่อรักษาตัวตนที่สอดคล้องกันข้ามเฟรม person_id เดียวกันในเฟรมต่างๆ หมายถึงบุคคลเดียวกันที่กำลังถูกติดตาม |
| `avg_confidence` | float64 | คะแนนความเชื่อมั่นในการตรวจจับเฉลี่ย (0.0-1.0) จาก keypoint ที่ถูกต้องทั้งหมดของบุคคลนี้ในเฟรมนี้ ค่าที่สูงกว่าบ่งบอกว่าการตรวจจับท่าทางน่าเชื่อถือมากกว่า |

**Keypoints ศีรษะ/ใบหน้า (5 จุด):**

| คอลัมน์ | ชนิดข้อมูล | คำอธิบาย |
|--------|-------|-------------|
| `nose_x` | float64 | พิกัด X (พิกเซล) ของจุด keypoint จมูก เป็นจุดอ้างอิงกลางใบหน้า |
| `nose_y` | float64 | พิกัด Y (พิกเซล) ของจุด keypoint จมูก |
| `nose_conf` | float64 | ความเชื่อมั่นในการตรวจจับ (0.0-1.0) สำหรับจุด keypoint จมูก |
| `left_eye_x` | float64 | พิกัด X (พิกเซล) ของจุด keypoint ตาซ้าย (จากมุมมองของบุคคลในภาพ) |
| `left_eye_y` | float64 | พิกัด Y (พิกเซล) ของจุด keypoint ตาซ้าย |
| `left_eye_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ตาซ้าย |
| `right_eye_x` | float64 | พิกัด X (พิกเซล) ของจุด keypoint ตาขวา (จากมุมมองของบุคคลในภาพ) |
| `right_eye_y` | float64 | พิกัด Y (พิกเซล) ของจุด keypoint ตาขวา |
| `right_eye_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ตาขวา |
| `left_ear_x` | float64 | พิกัด X (พิกเซล) ของจุด keypoint หูซ้าย |
| `left_ear_y` | float64 | พิกัด Y (พิกเซล) ของจุด keypoint หูซ้าย |
| `left_ear_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint หูซ้าย |
| `right_ear_x` | float64 | พิกัด X (พิกเซล) ของจุด keypoint หูขวา |
| `right_ear_y` | float64 | พิกัด Y (พิกเซล) ของจุด keypoint หูขวา |
| `right_ear_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint หูขวา |

**Keypoints ร่างกายส่วนบน (6 จุด):**

| คอลัมน์ | ชนิดข้อมูล | คำอธิบาย |
|--------|-------|-------------|
| `left_shoulder_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อไหล่ซ้าย จุดอ้างอิงหลักสำหรับการวิเคราะห์การเคลื่อนไหวแขน |
| `left_shoulder_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อไหล่ซ้าย |
| `left_shoulder_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ไหล่ซ้าย |
| `right_shoulder_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อไหล่ขวา |
| `right_shoulder_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อไหล่ขวา |
| `right_shoulder_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ไหล่ขวา |
| `left_elbow_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อข้อศอกซ้าย จุดกลางของแขนซ้าย |
| `left_elbow_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อข้อศอกซ้าย |
| `left_elbow_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อศอกซ้าย |
| `right_elbow_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อข้อศอกขวา |
| `right_elbow_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อข้อศอกขวา |
| `right_elbow_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อศอกขวา |
| `left_wrist_x` | float64 | พิกัด X (พิกเซล) ของข้อมือซ้าย จุดปลายสุดของแขนซ้าย (end effector) |
| `left_wrist_y` | float64 | พิกัด Y (พิกเซล) ของข้อมือซ้าย |
| `left_wrist_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อมือซ้าย |
| `right_wrist_x` | float64 | พิกัด X (พิกเซล) ของข้อมือขวา |
| `right_wrist_y` | float64 | พิกัด Y (พิกเซล) ของข้อมือขวา |
| `right_wrist_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อมือขวา |

**Keypoints ร่างกายส่วนล่าง (6 จุด):**

| คอลัมน์ | ชนิดข้อมูล | คำอธิบาย |
|--------|-------|-------------|
| `left_hip_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อสะโพกซ้าย จุดอ้างอิงลำตัวหลักสำหรับร่างกายส่วนล่าง |
| `left_hip_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อสะโพกซ้าย |
| `left_hip_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint สะโพกซ้าย |
| `right_hip_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อสะโพกขวา |
| `right_hip_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อสะโพกขวา |
| `right_hip_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint สะโพกขวา |
| `left_knee_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อเข่าซ้าย |
| `left_knee_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อเข่าซ้าย |
| `left_knee_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint เข่าซ้าย |
| `right_knee_x` | float64 | พิกัด X (พิกเซล) ของข้อต่อเข่าขวา |
| `right_knee_y` | float64 | พิกัด Y (พิกเซล) ของข้อต่อเข่าขวา |
| `right_knee_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint เข่าขวา |
| `left_ankle_x` | float64 | พิกัด X (พิกเซล) ของข้อเท้าซ้าย จุดปลายสุดของขาซ้าย (end effector) |
| `left_ankle_y` | float64 | พิกัด Y (พิกเซล) ของข้อเท้าซ้าย |
| `left_ankle_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อเท้าซ้าย |
| `right_ankle_x` | float64 | พิกัด X (พิกเซล) ของข้อเท้าขวา |
| `right_ankle_y` | float64 | พิกัด Y (พิกเซล) ของข้อเท้าขวา |
| `right_ankle_conf` | float64 | ความเชื่อมั่นในการตรวจจับสำหรับจุด keypoint ข้อเท้าขวา |

**คอลัมน์ป้ายกำกับ (Label):**

| คอลัมน์ | ชนิดข้อมูล | คำอธิบาย |
|--------|-------|-------------|
| `action` | object | ป้ายกำกับท่าทาง/การกระทำที่กำหนดตามช่วงเฟรม ประกอบด้วยชื่อแม่ไม้มวยไทย (เช่น "Mek_Khara_Lor_Kaew" - เมฆขราลอยแก้ว, "Kum_PaGUN_Poong_Hork" - กำปั้นป่วงหก) หรือ "Unknown" สำหรับเฟรมที่ไม่มีป้ายกำกับ |

### หมายเหตุการใช้งานสำคัญ

1. **ระบบพิกัด**: พิกัด `_x` และ `_y` ทั้งหมดอยู่ในหน่วยพิกเซลเทียบกับจุดกำเนิดของเฟรมวิดีโอ (มุมบนซ้าย) ค่า Y เพิ่มขึ้นเมื่อไปทางล่าง

2. **การกรองด้วยความเชื่อมั่น**: ใช้คอลัมน์ `_conf` เพื่อกรองการตรวจจับที่มีความเชื่อมั่นต่ำ ค่า 0 โดยทั่วไปหมายความว่า keypoint นั้นไม่ถูกตรวจจับ

3. **การวิเคราะห์ Time Series**: จัดกลุ่มตาม `person_id` และเรียงลำดับตาม `frame_idx` หรือ `timestamp` เพื่อวิเคราะห์การเคลื่อนไหวของบุคคลแต่ละคนตามเวลา

4. **การจัดกลุ่มส่วนต่างๆ ของร่างกาย**:
   - ศีรษะ: nose, left_eye, right_eye, left_ear, right_ear
   - แขนซ้าย: left_shoulder, left_elbow, left_wrist
   - แขนขวา: right_shoulder, right_elbow, right_wrist
   - ขาซ้าย: left_hip, left_knee, left_ankle
   - ขาขวา: right_hip, right_knee, right_ankle
   - ลำตัว: left_shoulder, right_shoulder, left_hip, right_hip